In [ ]:
# This command imports the Gurobi functions and classes.
import os

os.environ['GRB_LICENSE_FILE'] = 'C:\\Users\\rohan\\gurobi.lic'

import gurobipy as gp
from gurobipy import GRB


import pandas as pd
from pylab import *
import matplotlib
import matplotlib.pyplot as plt



workdays = gp.tuplelist(['MON','TUES','WED','THURS','FRI',
                        'MON2','TUES2','WED2','THURS2','FRI2',
                        'MON3','TUES3','WED3','THURS3','FRI3',
                        'MON4','TUES4','WED4','THURS4','FRI4'])

mondays = gp.tuplelist(['MON','MON2','MON3','MON4'])
fridays = gp.tuplelist(['FRI','FRI2','FRI3','FRI4'])
saturadays = gp.tuplelist(['SAT','SAT2','SAT3','SAT4'])

weekends = gp.tuplelist(['SAT','SUN','SAT2','SUN2','SAT3','SUN3','SAT4','SUN4'])

alldays = gp.tuplelist(['MON','TUES','WED','THURS','FRI','SAT','SUN',
                        'MON2','TUES2','WED2','THURS2','FRI2','SAT2','SUN2',
                        'MON3','TUES3','WED3','THURS3','FRI3','SAT3','SUN3',
                        'MON4','TUES4','WED4','THURS4','FRI4','SAT4','SUN4']
                      )


shifts, shiftTime = gp.multidict({
    'W6':6,
    'W7':7,
    'W8':8,
    'W9':9,
    'W10':10})




m = gp.Model("sleeptime")



availability = []
for day in alldays:
    for w in shifts:
        availability.append((day,w))

gp_availability = gp.tuplelist(availability)

x = m.addVars(gp_availability, vtype=GRB.BINARY, name="x")


shift_reqmts = m.addConstrs((x.sum(day,'*') == 1 for day in workdays), name='shiftRequirement')
shift_reqmts2 = m.addConstrs((x.sum(day,'*') == 0 for day in weekends), name='shiftRequirement2')

# slacks = m.addVars(shifts, name="Slack")
# slack_reqmts = m.addConstrs((slacks[s] == starTime[s] for s in schedules), name='shiftRequirement')

sleeps, sleepTime = gp.multidict({
    'S0':24,
    'S1':25,
    'S2':26,
    'S3':27,
    'S23':23,
    'S22':22,
    'S21':21})


naps = []
for day in alldays:
    for w in sleeps:
        naps.append((day,w))

gp_naps = gp.tuplelist(naps)


y = m.addVars(gp_naps, vtype=GRB.BINARY, name="y")


sleep_reqmts = m.addConstrs((y.sum(day,'*') == 1 for day in alldays), name='sleepRequirement')

mod_work_time = {}
mod_sleep_time = {}
hour=0
for day in alldays:
    
    for s in shifts:
        mod_work_time[day,s] =  x[day,s]*shiftTime[s]+x[day,s]*hour
        
    
    for n in sleeps:
        mod_sleep_time[day,n] = y[day,n]*sleepTime[n]+y[day,n]*hour
        
    hour+=24
    
_,gp_mod_sleep_time = gp.multidict(mod_sleep_time)
_,gp_mod_work_time = gp.multidict(mod_work_time)

# Define Losses

In [1]:
#difference between sleeps
gp_tbss = []

for day in range(len(alldays)-1):
    i = alldays[day]
    j = alldays[day+1]
        
    tmp = gp_mod_sleep_time.sum(j,'*')-gp_mod_sleep_time.sum(i,'*')
    
    #decrease the time between sleeps if weekends
    if j not in weekends:
        tmp=tmp*-1.0
    
    gp_tbss.append(tmp)

gp_tbww = []
for day in range(len(alldays)-1):

    i = alldays[day]
    j = alldays[day+1]
    

    tmp = -1*(gp_mod_work_time.sum(j,'*')-gp_mod_work_time.sum(i,'*'))
    
    #ignore the value between work on fridays and monday
    if j in weekends:
        tmp=tmp*0.0
        
    if j in mondays:
        tmp=tmp*0.0
    
    gp_tbww.append(tmp)
    
gp_tbws = []
for day in range(len(alldays)-1):

    i = alldays[day]
    j = alldays[day+1]

    tmp = (gp_mod_work_time.sum(j,'*')-gp_mod_sleep_time.sum(i,'*'))
    
    #ignore time between sleep and work on weekends
    if j in weekends:
        tmp=tmp*0.0
        
    #make sure I sleep enough on Sunday
    if j in mondays:
        tmp=tmp*10.0
    
    gp_tbws.append(tmp)

Academic license - for non-commercial use only - expires 2021-12-01
Using license file C:\Users\rohan\gurobi.lic


In [2]:

m.params.OutputFlag = 1
m.params.timelimit = 6000
# m.params.mipgap = 0.000001

TSS = gp.quicksum(gp_tbss)
TWW = gp.quicksum(gp_tbww)
TWS = gp.quicksum(gp_tbws)

m.ModelSense = GRB.MAXIMIZE

# m.setObjective(TSS)
m.setObjectiveN(TWW, index=3, priority=3, name='CONST')
m.setObjectiveN(TSS, index=2, priority=2, name='REST')
m.setObjectiveN(TWS, index=0, priority=1, name='EFFICIENCY')

m.optimize()

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter timelimit to 6000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 56 rows, 336 columns and 336 nonzeros
Model fingerprint: 0x0d67a95b
Variable types: 0 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 8 rows and 40 columns
Presolve time: 0.0

# Work Assignments

In [3]:
assignments = dict()
for [w, s] in availability:
    if x[w, s].x == 1:
        if w in assignments:
            assignments[w].append(s)
        else:
            assignments[w] = [s]

assignments

{'MON': ['W10'],
 'TUES': ['W10'],
 'WED': ['W10'],
 'THURS': ['W10'],
 'FRI': ['W6'],
 'MON2': ['W10'],
 'TUES2': ['W10'],
 'WED2': ['W10'],
 'THURS2': ['W10'],
 'FRI2': ['W6'],
 'MON3': ['W10'],
 'TUES3': ['W10'],
 'WED3': ['W10'],
 'THURS3': ['W10'],
 'FRI3': ['W6'],
 'MON4': ['W10'],
 'TUES4': ['W10'],
 'WED4': ['W10'],
 'THURS4': ['W10'],
 'FRI4': ['W6']}

# Sleep Assignments

In [4]:
assignments = dict()
for [w, s] in naps:
    if y[w, s].x == 1:
        if w in assignments:
            assignments[w].append(s)
        else:
            assignments[w] = [s]
            
assignments

{'MON': ['S3'],
 'TUES': ['S21'],
 'WED': ['S21'],
 'THURS': ['S21'],
 'FRI': ['S21'],
 'SAT': ['S0'],
 'SUN': ['S3'],
 'MON2': ['S21'],
 'TUES2': ['S21'],
 'WED2': ['S21'],
 'THURS2': ['S21'],
 'FRI2': ['S21'],
 'SAT2': ['S0'],
 'SUN2': ['S3'],
 'MON3': ['S21'],
 'TUES3': ['S21'],
 'WED3': ['S21'],
 'THURS3': ['S21'],
 'FRI3': ['S21'],
 'SAT3': ['S0'],
 'SUN3': ['S3'],
 'MON4': ['S21'],
 'TUES4': ['S21'],
 'WED4': ['S21'],
 'THURS4': ['S21'],
 'FRI4': ['S21'],
 'SAT4': ['S0'],
 'SUN4': ['S3']}